## Notebook to develop a Twitter _searchAPI_ collector

functions to save to and load from mongo
functions tho exeute the queries to the API

METHODS WE WANT TO USE IN PROJECT:

- [GET search/tweets](https://dev.twitter.com/rest/reference/get/search/tweets)
- [GET statuses/user_timeline](https://dev.twitter.com/rest/reference/get/statuses/user_timeline)


## Notebook to develop a Twitter _streamingAPI_ collector

METHODS WE WANT TO USE IN PROJECT:
- [GET statuses/sample](https://dev.twitter.com/streaming/reference/get/statuses/sample)

---

# Mixed use to retrive data faster
How do we find keywords related tweets faster?

We use a mix of the SEARCH and STREAMING APIs:
1. Retrive as meany tweets as possible in the past (past week) using the SEARCH API
2. Continue the search using the STREAMING API until the number of desired tweets associated with the keyword is retrived

In [2]:
import twitter
import urllib2
import pymongo
import sys
import json
import re
import os

In [15]:
# manually_selected_last = ['drug', 'heroin', 'marijuana', 'nicotine', 'quit smoking', 'smoke anymore', 'smokeless', 'smoker', 'smoking habit', 'cig', 'cigarette advert', 'cigarette brand', 'cigarette pack', 'malboro', 'ecig', 'electronic cigarette', 'vaping', 'vaporizer', 'hookah', 'shisha', 'tobacco control', 'tobacco illegal', 'tobacco industry', 'tobacco market', 'tobacco product']
# only unigram
manually_selected_last = ['drug', 'marijuana', 'nicotine', 'tobacco', 'smoking', 'smoke', 'smoker', 'cig', 'cigarette', 'ecig', 'vaping', 'vaporizer', 'shisha']
print len(manually_selected_last)
# MANUALLY INPUT EACH KEY-WORD

# Use the SEARCHER to gather as many tweets as possible in the past
# Use the STREAMER to arrive at 2000 tweets per keyword

13


In [11]:
# SEARCHER
# credentials: "coffee1"
########################################################

def oauth_login():

    CONSUMER_KEY = 'cOExq4T3uYu9CbdqmMUukGVqV'
    CONSUMER_SECRET = 'u9WCLUaWCLyILtHNGDAMmowxoQseQenyW0avsXhuz8NkPgLHMv'
    OAUTH_TOKEN = '4061210361-uHZhAHfTim8nnyyCWeZXMfLbypbq3Z2jgGcTmhu'
    OAUTH_TOKEN_SECRET = 'nbC4YzeS7vBBBcOWyYgr4msKdLgfRznFQAon1ji2lJn2F'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
     CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)

    print >> sys.stderr, 'Succesfully connected to TwitterAPI ... ... '

    return twitter_api


def twitter_search(twitter_api, q, max_results=2000, **kw):
    
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    
    statuses = search_results['statuses']
    
    for _ in range(179): # 180 max number of query we can issue in 15 minutes (1 it is outside the loop)
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError, e: # No more results when next_results doesn't exist
            print "Length of statuses", len(statuses) 
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)

        statuses.extend(search_results['statuses'])
        
        if len(statuses) > max_results: 
            break
        
    return statuses


def save_to_mongo(data, mongo_db, mongo_db_coll, **mongo_conn_kw):
    try:
        mongo_db_coll.insert_one(data)
        print "Saving tweet"
    except:
        print "Error message from MongoDB, skipping tweet and continuing."
        




if __name__ == "__main__":

    # search keyword ONE by ONE, gather 2000 tweets for each


    ##############################################
    ##############################################
    #   Stream parameters: just modify these, do NOT touch the rest
    ##############################################
    ##############################################

    client = pymongo.MongoClient()
    db = client.stream   # DATABASE_NAME
    collection = db.get_hashtags     # COLLECTION_NAME

    # search keyword ONE by ONE, gather 2000 tweets for each
    keyword = 'shisha'

    TAGS = keyword  
    TAGS_LIST = [keyword]

    ##############################################
    ##############################################
    ##############################################

    twitter_api = oauth_login()
    q = keyword
    results = twitter_search(twitter_api, q, lang='en')

    for tweet_doc in results:

        try:
            # 2 small modificatio to the tweet as it is gathered from API

            # 1. Add the tags of the search, to know from which search query the document has been obtained
            tweet_doc['keyword'] = TAGS_LIST  

            # 2. Substitue the defoult _id assigned by Mongo with the tweet id to prevent the same tweet to be inserted more than once
            tweet_doc['_id'] = tweet_doc["id"]
            del tweet_doc['id']

            save_to_mongo(tweet_doc, db, collection)

            print "\n %s \n" % tweet_doc['text']

            documents_in_collection = collection.find({"keyword": keyword}).count()

            if documents_in_collection >= 2000:

                print "Stremed 2000 tweets associated with keyword: \"%s\"" % keyword
                break

        except KeyError:
            print "KeyError, skipping tweet and continuing."
            continue

Succesfully connected to TwitterAPI ... ... 


Length of statuses 897
Error message from MongoDB, skipping tweet and continuing.

 RT @Abunaah: Ku tsaya Guys ✋ Tickets for MIC Concert Bauchi is out! Grab yours @ Traffic shisha spot or call 08057059007 https://t.co/jdJ61… 

Error message from MongoDB, skipping tweet and continuing.

 RT @Abunaah: Tickets for MIC Concert now on sale @ Traffic Shisha Shop  Or 
Call  08126278353
For Reservations, call 08057059007 https://t.… 

Error message from MongoDB, skipping tweet and continuing.

 Smoke with us in Style! I https://t.co/odMUGWzYZu #SHISHA https://t.co/UYIANuA2Uq 

Error message from MongoDB, skipping tweet and continuing.

 RT @EmilyNHurley: Videoing yourself whilst smoking a shisha is the single most gayest thing you can do 

Error message from MongoDB, skipping tweet and continuing.

 Majority of Shisha bars operating adhere to the law. Does the Shisha bar you visit? https://t.co/duaHXIrSAQ https://t.co/b1EgxEmvw2 

Error message from MongoDB, skipping tweet and continuing.

 Sh

In [12]:
# STREAMER
# credentials: "searcher_nv_0"
########################################################

def oauth_login():

    CONSUMER_KEY = 'FqWhSxd57N9zgn3sr2Jm1cUYg'
    CONSUMER_SECRET = 'LpRHxDlWsGp7uuwDrpouVcOrL6dTsx0qPccKxBXbvlENSyRaHw'
    OAUTH_TOKEN = '4061210361-mh8Hn8pvOabKNfj2dohmk7gBknzWQsARAo2A2lk'
    OAUTH_TOKEN_SECRET = 'XHfTHWN3T6oqgFKTfo4fA2WylPxzi44QDNYdoRDsE868c'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
     CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)

    print >> sys.stderr, 'Succesfully connected to TwitterAPI ... ... '

    return twitter_api



def save_to_mongo(data, mongo_db, mongo_db_coll, **mongo_conn_kw):
    try:
        mongo_db_coll.insert_one(data)
        # print "Saving tweet"
    except:
        print "Error message from MongoDB. Continuing."

        
        
        



if __name__ == "__main__":

    # search keyword ONE by ONE, gather 2000 tweets for each

    ##############################################
    ##############################################
    #   Stream parameters: just modify these, do NOT touch the rest
    ##############################################
    ##############################################

    client = pymongo.MongoClient()
    db = client.stream   # DATABASE_NAME
    collection = db.get_hashtags     # COLLECTION_NAME

    # search keyword ONE by ONE, gather 2000 tweets for each
    keyword = 'shisha'

    TAGS = keyword  
    TAGS_LIST = [keyword]

    ##############################################
    ##############################################
    ##############################################

    twitter_api = oauth_login()
    twitter_stream = twitter.TwitterStream(auth=twitter_api.auth)
    stream = twitter_stream.statuses.filter(track=TAGS, language='en')
    #stream = twitter_stream.statuses.sample(language='en') # stream a RANDOM sample
    print "Succesfully created a stream instance ... ... \n"
        #print "Filtering the public timeline for track="%s" \t \t " % (TAGS)

    for tweet_doc in stream:

        try:
            # 2 small modificatio to the tweet as it is gathered from API

            # 1. Add the tags of the search, to know from which search query the document has been obtained
            tweet_doc['keyword'] = TAGS_LIST  

            # 2. Substitue the defoult _id assigned by Mongo with the tweet id to prevent the same tweet to be inserted more than once
            tweet_doc['_id'] = tweet_doc["id"]
            del tweet_doc['id']

            save_to_mongo(tweet_doc, db, collection)

            print "\n %s \n" % tweet_doc['text']

            documents_in_collection = collection.find({"keyword": keyword}).count()

            if documents_in_collection >= 28000:

                print "Stremed 2000 tweets associated with keyword: \"%s\"" % keyword
                break
        
        except KeyError:
            print "KeyError, skipping tweet and continuing."
            continue

Succesfully connected to TwitterAPI ... ... 


Succesfully created a stream instance ... ... 


 No thanks. (with Adam, Arief , and Ilham Yusuf at Furdan Shisha And Cafe) — https://t.co/b2PqtWCVpe 


 EN DIRECTO en #Periscope: Shisha https://t.co/49toOLW3WD 


 Thank you bro Shisha! See ya at Mall of Medini #medinilive #letsgotomedini https://t.co/k1qRtk9hmx 


 Best Shisha in Hong Kong @?????????? #secrets #legit #boss #insiderknowledge @ Central District,… https://t.co/RIeDtaUE61 


 chillin (@ Family Shisha in Ipoh) https://t.co/cQIAOkjecW 


 RT @AndreGray7: Shisha times http://t.co/w6EiwErZv2 


 RT @Eudaimonalisa: Is it true that in Bradford Uni there's shisha pipes in lectures 


 RT @AndreGray7: Shisha times http://t.co/w6EiwErZv2 


 I'm at Saudagar Shisha Zam-Zam Aft Corner in Johor Bahru https://t.co/nAbRdu4CNP 


 RT @AndreGray7: Shisha times http://t.co/w6EiwErZv2 


 RT @AndreGray7: Shisha times http://t.co/w6EiwErZv2 


 RT @AndreGray7: Shisha times http://t.co/w6EiwErZv2 


 RT @AndreGray7: Shisha times http://t.co/

KeyboardInterrupt: 